# Exercícios: Regressão

In [ ]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

Para estes exercícios, será usado o conjunto de dados [California Housing](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html):

In [ ]:
from sklearn.datasets import fetch_california_housing

ds = fetch_california_housing()

# Convert the data to a pandas DataFrame
df = pd.DataFrame(ds.data, columns=ds.feature_names)
df['target'] = ds.target

1. Dedique algum tempo a rever e a comprender as várias colunas do conjunto de dados e o problema a abordar, tendo também em conta a descrição que pode encontrar no [website](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html).

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.corr()

2. Particione o conjunto de dados de forma a que 80% seja para treino e 20% para teste.

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

3. Usando uma abordagem de validação cruzada sobre o conjunto de treino, explore o uso de diferentes algoritmos de regressão e analise as diferenças entre os resultados obtidos. Para as abordagens com menor desempenho, pense em estratégias de processamento de dados que poderiam ser exploradas para melhorar esse desempenho.

In [ ]:
regressors = {
    'Linear Regression': LinearRegression(),
    'k-NN': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(n_estimators=100),
    'SVM': SVR(kernel='linear', max_iter=10000),
    'MLP': MLPRegressor(hidden_layer_sizes=(8,4), max_iter=20000)
}

In [ ]:
cv_scores = pd.Series({
    name: np.mean(cross_val_score(regressor, df_train.drop(columns=['target']), df_train['target'], cv=5))
    for name, regressor in regressors.items()
})

cv_scores

4. Entre as abordagens exploradas no ponto 3, selecione a melhor. Use essa abordagem para treinar um regressor no conjunto de treino. Avalie o desempenho desse classificador no conjunto de teste usando as seguintes métricas: erro quadrático médio (MSE), erro absoluto médio (MAE) e coeficiente de determinação (R<sup>2</sup>).

In [ ]:
best = cv_scores.sort_values(ascending=False).index[0]
best_regressor = regressors[best]
best_regressor.fit(df_train.drop(columns=['target']), df_train['target'])

print(f'### {best} ###')

y_pred = best_regressor.predict(df_test.drop(columns=['target']))

print('r2: {}'.format(r2_score(df_test['target'], y_pred)))
print('mse: {}'.format(mean_squared_error(df_test['target'], y_pred)))
print('mae: {}'.format(mean_absolute_error(df_test['target'], y_pred)))

5. Ordene os exemplos do conjunto de teste por ordem decrescente do erro da previsão do regressor e verifique se existe algum padrão relevante. 

In [ ]:
df_error = df_test.copy()
df_error['error'] = np.abs(df_error['target'] - y_pred)

df_error.sort_values('error', ascending=False).head(20)